# CatBoost Exercise - Costa Rican Household Poverty Level Prediction
This notebook assumes you are already familiar with the basics of catboost and are ready for some experiments and hyperparameter tuning. 

We will focus here on the Kaggle task "Costa Rican Household Poverty Level Prediction". If you haven't gone through the [walkthrough notebook](a-complete-introduction-and-walkthrough.ipynb) yet, this is the time to do so. The output of the notebook is a csv file named `final.csv`, and is expected to be located under the `other_data` folder.

This notebook will guide you through the important parts of the exercise, but will leave a lot of room for you to experiment and try different things.

Good luck!

## Imports
Add all of your imports here

In [1]:
!pip install catboost
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [3]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import f1_score, make_scorer
from catboost import CatBoostClassifier, Pool, cv

# Set a few plotting defaults
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['font.size'] = 8
plt.rcParams['patch.edgecolor'] = 'k'

## Data Loading

First, let's recall the different columns we had in our final dataset. This can be useful for defining later which features are categorical and which are not. Remember that there are also aggregation columns (min, max, std, etc.) but they are not mentioned here for simplicity. One can retrieve them easily by simple string matching like so: `<column_name>-<stat>`, for each desired column and aggregation statistic (one of the following: `['min', 'max', 'sum', 'count', 'std', 'range_']`)
* Remember that we dropped some of the columns, so you might have to filter column names that do not appear in the final dataset.

In [4]:
id_ = ['Id', 'idhogar', 'Target']

ind_bool = ['v18q', 'dis', 'male', 'female', 'estadocivil1', 'estadocivil2', 'estadocivil3', 
            'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 
            'parentesco1', 'parentesco2',  'parentesco3', 'parentesco4', 'parentesco5', 
            'parentesco6', 'parentesco7', 'parentesco8',  'parentesco9', 'parentesco10', 
            'parentesco11', 'parentesco12', 'instlevel1', 'instlevel2', 'instlevel3', 
            'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 
            'instlevel9', 'mobilephone', 'rez_esc-missing']

ind_ordered = ['rez_esc', 'escolari', 'age']

hh_bool = ['hacdor', 'hacapo', 'v14a', 'refrig', 'paredblolad', 'paredzocalo', 
           'paredpreb','pisocemento', 'pareddes', 'paredmad',
           'paredzinc', 'paredfibras', 'paredother', 'pisomoscer', 'pisoother', 
           'pisonatur', 'pisonotiene', 'pisomadera',
           'techozinc', 'techoentrepiso', 'techocane', 'techootro', 'cielorazo', 
           'abastaguadentro', 'abastaguafuera', 'abastaguano',
            'public', 'planpri', 'noelec', 'coopele', 'sanitario1', 
           'sanitario2', 'sanitario3', 'sanitario5',   'sanitario6',
           'energcocinar1', 'energcocinar2', 'energcocinar3', 'energcocinar4', 
           'elimbasu1', 'elimbasu2', 'elimbasu3', 'elimbasu4', 
           'elimbasu5', 'elimbasu6', 'epared1', 'epared2', 'epared3',
           'etecho1', 'etecho2', 'etecho3', 'eviv1', 'eviv2', 'eviv3', 
           'tipovivi1', 'tipovivi2', 'tipovivi3', 'tipovivi4', 'tipovivi5', 
           'computer', 'television', 'lugar1', 'lugar2', 'lugar3',
           'lugar4', 'lugar5', 'lugar6', 'area1', 'area2', 'v2a1-missing']

hh_ordered = [ 'rooms', 'r4h1', 'r4h2', 'r4h3', 'r4m1','r4m2','r4m3', 'r4t1',  'r4t2', 
              'r4t3', 'v18q1', 'tamhog','tamviv','hhsize','hogar_nin',
              'hogar_adul','hogar_mayor','hogar_total',  'bedrooms', 'qmobilephone']

hh_cont = ['v2a1', 'dependency', 'edjefe', 'edjefa', 'mebaneduc', 'overcrowding']

sqr_ = ['SQBescolari', 'SQBage', 'SQBhogar_total', 'SQBedjefe', 
        'SQBhogar_nin', 'SQBovercrowding', 'SQBdependency', 'SQBmeaned', 'agesq']

cat_cols = ['elec', 'female_head' ]
cat_cols_with_order = ['walls', 'roof', 'floor', 'walls+roof+floor', 'inst', 'tech']

Load the preprocessed dataset, extract train and test splits (remember that the test split doesn't have any labels, you must sumbit a solution to Kaggle in order to get your performance on the test set). 

In [6]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 1A39-E879

 Directory of C:\Users\yyahe\Google Drive\ML training\CatBoost part

10/06/2020  07:57    <DIR>          .
10/06/2020  07:57    <DIR>          ..
10/06/2020  07:57    <DIR>          .ipynb_checkpoints
15/03/2020  18:32        10,187,726 1.jpg
15/03/2020  18:32        11,118,219 2.jpg
15/03/2020  18:32        11,017,917 3.jpg
15/03/2020  18:32        11,682,306 4.jpg
15/03/2020  18:32        11,705,686 5.jpg
07/05/2020  10:18         2,313,477 CatBoost vs. Light GBM vs. XGBoost - Towards Data Science.pdf
18/05/2020  10:58            22,288 catboost_exercise.docx
10/06/2020  07:57            48,739 catboost_solution.ipynb
21/05/2020  13:36            18,485 catboosting.ipynb
26/05/2020  11:55            42,251 gridcv_results.csv
07/05/2020  10:18           642,613 Hyperparameter Tuning - O'Reilly Media.pdf
21/05/2020  13:31    <DIR>          input
21/05/2020  13:31         1,320,337 input-20200521T103034Z-001.zip
21/05/

In [8]:
final = pd.read_csv('other_data/final.csv')

# Labels for training
train_labels = np.array(list(final[final['Target'].notnull()]['Target'].astype(np.uint8)))
test_ids = list(final.loc[final['Target'].isnull(), 'idhogar'])

# Extract the training data
train_set = final[final['Target'].notnull()].drop(columns = ['Id', 'idhogar', 'Target'])
test_all_data = final[final['Target'].isnull()]
submission_base = test_all_data[['Id', 'idhogar']].copy()
test_set = test_all_data.drop(columns = ['Id', 'idhogar', 'Target'])

### Data Preparation
Because we don't have a lot of data in the training set, it will be best to use cross validation for evaluation instead of splitting the training set into a single train-dev split. You also want your different experiments to be comparable. Therefore, you should **create a single set of CV splits of the data which you will use for all of your experiments**. Think about the time vs variance tradeoff when deciding how many splits you want to have (5 is reasonable). **Remember to shuffle and stratify!**

* Note: If you don't know yet what is cross validation, why shuffling the data is important, or what it means to stratify the data when splitting it into train-dev-test splits, this is a great time to do so. It's important!

In [9]:
strkfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

In [14]:
def macro_f1_score(labels, predictions):
    # Reshape the predictions as needed
    predictions = predictions.reshape(len(np.unique(labels)), -1 ).argmax(axis = 0)
    
    metric_value = f1_score(labels, predictions, average = 'macro')
    
    # Return is name, value, is_higher_better
    return 'macro_f1', metric_value, True

In [15]:
def cross_val(model, features, labels, test_features, strkfold, cat_features_indices=None):
    nfolds = strkfold.n_splits
    feature_names = list(features.columns)
    # Hold all the predictions from each fold
    predictions = pd.DataFrame()
    importances = np.zeros(features.shape[1])
    
    # Convert to arrays for indexing
    features = np.array(features)
    test_features = np.array(test_features)
    labels = np.array(labels).reshape((-1 ))
    
    valid_scores = []
    
    # Iterate through the folds
    for i, (train_indices, valid_indices) in enumerate(strkfold.split(features, labels)):
        
        # Dataframe for fold predictions
        fold_predictions = pd.DataFrame()
        
        # Training and validation data
        X_train = features[train_indices]
        X_valid = features[valid_indices]
        y_train = labels[train_indices]
        y_valid = labels[valid_indices]
        
        model.fit(X_train, y_train,
                  cat_features=cat_features_indices,
                  eval_set=(X_valid, y_valid))
        
        # Record the validation fold score
        y_pred = model.predict(X_valid)
        valid_scores.append(f1_score(y_valid, y_pred, average = 'macro'))
        
        # Make predictions from the fold as probabilities
        fold_probabilitites = model.predict_proba(test_features)
        
        # Record each prediction for each class as a separate column
        for j in range(4):
            fold_predictions[(j + 1)] = fold_probabilitites[:, j]
            
        # Add needed information for predictions 
        fold_predictions['idhogar'] = test_ids
        fold_predictions['fold'] = (i+1)
        
        # Add the predictions as new rows to the existing predictions
        predictions = predictions.append(fold_predictions)
        
        # Feature importances
        importances += model.feature_importances_ / nfolds   
        
        # Display fold information
        display(f'Fold {i + 1}, Validation Score: {round(valid_scores[i], 5)}, Estimators Trained: {model.best_iteration_}')
        
    # Feature importances dataframe
    feature_importances = pd.DataFrame({'feature': feature_names,
                                        'importance': importances})

    valid_scores = np.array(valid_scores)
    display(f'cross validation F1-score: {round(valid_scores.mean(), 5)} with std: {round(valid_scores.std(), 5)}.')
    
    # Average the predictions over folds
    predictions = predictions.groupby('idhogar', as_index = False).mean()
    
    # Find the class and associated probability
    predictions['Target'] = predictions[[1, 2, 3, 4]].idxmax(axis = 1)
    predictions['confidence'] = predictions[[1, 2, 3, 4]].max(axis = 1)
    predictions = predictions.drop(columns = ['fold'])
    
    # Merge with the base to have one prediction for each individual
    submission = submission_base.merge(predictions[['idhogar', 'Target']], on = 'idhogar', how = 'left').drop(columns = ['idhogar'])
        
    # Fill in the individuals that do not have a head of household with 4 since these will not be scored
    submission['Target'] = submission['Target'].fillna(4).astype(np.int8)
    
    # return the submission and feature importances along with validation scores
    return submission, feature_importances, valid_scores, predictions

In [16]:
def plot_confidence_by_target(predictions):
    predictions = predictions.groupby('idhogar', as_index = False).mean()

    # Find the class and associated probability
    predictions['Target'] = predictions[[1, 2, 3, 4]].idxmax(axis = 1)
    predictions['confidence'] = predictions[[1, 2, 3, 4]].max(axis = 1)
#     predictions = predictions.drop(co?lumns = ['fold'])

    # Plot the confidence by each target
    plt.figure(figsize = (10, 6))
    sns.boxplot(x = 'Target', y = 'confidence', data = predictions);
    plt.title('Confidence by Target');

    plt.figure(figsize = (10, 6))
    sns.violinplot(x = 'Target', y = 'confidence', data = predictions);
    plt.title('Confidence by Target');

In [17]:
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Oranges):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.figure(figsize = (10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, size = 24)
    plt.colorbar(aspect=4)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, size = 14)
    plt.yticks(tick_marks, classes, size = 14)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    
    # Labeling the plot
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), fontsize = 20,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
        
    plt.grid(None)
    plt.tight_layout()
    plt.ylabel('True label', size = 18)
    plt.xlabel('Predicted label', size = 18)
    
    

## Experiments
You will now expriment with various catboost models. You will go through the following steps:
1. Training a basic model for an initial baseline and to warm up
2. Selecting categorical features
3. Using class weights to compensate for imbalanced data
4. Performing grid search over some hypreparameter space
5. Performing random search over some hyperparameter space
6. Optional: performing bayesian hyperparameter optimization using the hyperopt library

Note: in some of the experiments you might want to parallelize the training of different models using sklearn njobs argument. In this case, you must run your code in a .py script. If you indeed do this, save the output of each experiment to a csv file, load it here and present the results in a meaningful way (plots etc.).

### Basic Model
Build a catboost model using the default parameters, and don't define any categorical features explicitly. 

In [19]:
default_model = CatBoostClassifier(
    random_seed=42,
#     verbose=200,
    logging_level='Silent',
    custom_loss='TotalF1:average=Macro',
#     class_weights = [0.25, 0.25, 0.25, 0.25],
    eval_metric='TotalF1:average=Macro'
)

# scorer = make_scorer(f1_score, greater_is_better=True, average = 'macro')
# cv_score = cross_val_score(default_model, train_set, train_labels, cv=strkfold, scoring = scorer)

_, feature_importances, valid_scores, _ = cross_val(default_model, train_set, 
                                                    train_labels, test_set, strkfold)

'Fold 1, Validation Score: 0.39457, Estimators Trained: 234'

'Fold 2, Validation Score: 0.40762, Estimators Trained: 387'

'Fold 3, Validation Score: 0.40849, Estimators Trained: 267'

'Fold 4, Validation Score: 0.44315, Estimators Trained: 429'

'Fold 5, Validation Score: 0.37856, Estimators Trained: 485'

'cross validation F1-score: 0.40648 with std: 0.02132.'

In [20]:
feature_importances.sort_values(by='importance', ascending=False).head(10)

,feature,importance
97,phones-per-capita,3.827971
69,meaneduc,3.554440
66,dependency,3.102202
204,age-min,2.582773
207,age-std,2.423274
206,age-sum,2.308501
205,age-max,2.124288
212,escolari/age-min,2.004123
200,escolari-max,1.996252
94,walls+roof+floor,1.658207


### Categorical Features
As you should already know, catboost has some nice support for categorical features. Experiment with this a bit, and tell catboost which of your features are categorical. Also, play with the `one_hot_max_size` hyperparameter. Understand what this hyperparameter is and how it affects the underlying algorithm.

In [21]:
non_binary_feat = np.where(train_set.apply(lambda x: x.nunique() == 2) == False)[0]
non_float_feat = np.where(train_set.dtypes != np.float)[0]

print(f'# of non binary: {len(non_binary_feat)}, # of non float: {len(non_float_feat)}')


# of non binary: 109, # of non float: 187


In [ ]:
# cat_features_indices = np.intersect1d(non_binary_feat, non_float_feat)
cat_features_indices = non_float_feat

model_with_cat = CatBoostClassifier(
#     cat_features=cat_features_indices,
    random_seed=42,
    one_hot_max_size = 2,
#     verbose=200,
    logging_level='Silent',
    custom_loss='TotalF1:average=Macro',
    class_weights = [0.25, 0.25, 0.25, 0.25],
    eval_metric='TotalF1:average=Macro'
)


_, feature_importances, valid_scores, _ = cross_val(model_with_cat, train_set, 
                                                    train_labels, test_set, strkfold, cat_features_indices)

In [ ]:
cat_features_indices = np.intersect1d(non_binary_feat, non_float_feat)
# cat_features_indices = non_float_feat

model_with_cat = CatBoostClassifier(
#     cat_features=cat_features_indices,
    random_seed=42,
    one_hot_max_size = 2,
#     verbose=200,
    logging_level='Silent',
    custom_loss='TotalF1:average=Macro',
    class_weights = [0.25, 0.25, 0.25, 0.25],
    eval_metric='TotalF1:average=Macro'
)


_, feature_importances, valid_scores, _ = cross_val(model_with_cat, train_set,
                                                    train_labels, test_set, strkfold, cat_features_indices)

In [ ]:
feature_importances.sort_values(by='importance', ascending=False).head(10)

In [ ]:
# cat_features_indices = np.intersect1d(non_binary_feat, non_float_feat)
cat_features_indices = non_float_feat

model_with_cat = CatBoostClassifier(
#     cat_features=cat_features_indices,
    random_seed=42,
    one_hot_max_size=3,
#     verbose=200,
    logging_level='Silent',
    custom_loss='TotalF1:average=Macro',
    class_weights = [0.25, 0.25, 0.25, 0.25],
    eval_metric='TotalF1:average=Macro'
)

_, feature_importances, valid_scores, _ = cross_val(model_with_cat, train_set,
                                                    train_labels, test_set, strkfold, cat_features_indices)

### Class Weights
As you have seen in the dataset walkthrough notebook, our data is very imbalanced (in terms of label frequency). One popular way to deal with imbalanced data is to weight each sample proportionaly to its inverse label frequency. Experiment with the `class_weights` hyperpameter: does the weighting improve the results? try a few different weighting schemes and see how they affect the model's performance.

In [22]:
unique, counts = np.unique(train_labels, return_counts=True)
label_freq = len(train_labels)/counts/4
label_freq = np.flip(label_freq).tolist()
print(label_freq)

[0.3803735926305015, 2.093661971830986, 1.6815610859728507, 3.347972972972973]


In [ ]:
# cat_features_indices = np.intersect1d(non_binary_feat, non_float_feat)
cat_features_indices = non_float_feat

model_with_cat = CatBoostClassifier(
#     cat_features=cat_features_indices,
    random_seed=42,
    one_hot_max_size=2,
    verbose=500,
#     logging_level='Silent',
    custom_loss='TotalF1:average=Macro',
    class_weights = label_freq,
    eval_metric='TotalF1:average=Macro'
)

_, feature_importances, valid_scores, _ = cross_val(model_with_cat, train_set,
                                                    train_labels, test_set, strkfold, cat_features_indices)

In [ ]:
# cat_features_indices = np.intersect1d(non_binary_feat, non_float_feat)
cat_features_indices = non_float_feat

model = CatBoostClassifier(
#     cat_features=cat_features_indices,
    random_seed=42,
    use_best_model=True,
    one_hot_max_size=2,
#     verbose=990,
    logging_level='Silent',
    custom_loss='TotalF1:average=Macro',
    class_weights = label_freq,
    eval_metric='TotalF1:average=Macro'
)

_, feature_importances_with_cat2, valid_scores_with_cat2, predictions_with_cat2 = cross_val(model, train_set, train_labels, 
                                                                                            test_set, strkfold, cat_features_indices)
plot_confidence_by_target(predictions_with_cat2)

In [ ]:
unique, counts = np.unique(train_labels, return_counts=True)
label_freq = len(train_labels)/counts/4
label_freq1 = np.flip(label_freq).tolist()

cat_features_indices = non_float_feat

model = CatBoostClassifier(
#     cat_features=cat_features_indices,
    random_seed=42,
    use_best_model=True,
    one_hot_max_size=2,
#     verbose=990,
    logging_level='Silent',
    custom_loss='TotalF1:average=Macro',
    class_weights = label_freq1,
    eval_metric='TotalF1:average=Macro'
)

_, feature_importances_with_cat2a, valid_scores_with_cat2a, predictions_with_cat2a = cross_val(model, train_set, train_labels, 
                                                                                            test_set, strkfold, cat_features_indices)
plot_confidence_by_target(predictions_with_cat2a)

In [ ]:
# cat_features_indices = np.intersect1d(non_binary_feat, non_float_feat)
cat_features_indices = non_float_feat

model = CatBoostClassifier(
#     cat_features=cat_features_indices,
    random_seed=42,
    use_best_model=True,
    one_hot_max_size=4,
#     verbose=990,
    logging_level='Silent',
    custom_loss='TotalF1:average=Macro',
    class_weights = label_freq,
    eval_metric='TotalF1:average=Macro'
)

_, feature_importances_with_cat4, valid_scores_with_cat4, predictions_with_cat4 = cross_val(model, train_set, train_labels, 
                                                                                            test_set, strkfold, cat_features_indices)
plot_confidence_by_target(predictions_with_cat4)

In [ ]:
# cat_features_indices = np.intersect1d(non_binary_feat, non_float_feat)
cat_features_indices = non_float_feat

model = CatBoostClassifier(
#     cat_features=cat_features_indices,
    random_seed=42,
    use_best_model=True,
    one_hot_max_size=3,
#     verbose=990,
    logging_level='Silent',
    custom_loss='TotalF1:average=Macro',
    class_weights = label_freq,
    eval_metric='TotalF1:average=Macro'
)

_, feature_importances_with_cat3, valid_scores_with_cat3, predictions_with_cat3 = cross_val(model, train_set, train_labels, 
                                                                                            test_set, strkfold, cat_features_indices)
plot_confidence_by_target(predictions_with_cat3)

In [ ]:
model = CatBoostClassifier(
    random_seed=42,
    use_best_model=True,
    one_hot_max_size=4,
#     verbose=990,
    logging_level='Silent',
    custom_loss='TotalF1:average=Macro',
    class_weights = label_freq,
    eval_metric='TotalF1:average=Macro'
)

_, feature_importances_without_cat, valid_scores_without_cat, predictions_without_cat = cross_val(
                                                                                            model, train_set, train_labels, test_set, strkfold)
plot_confidence_by_target(predictions_without_cat)

## Statistical Significance
When comparing the performance of different models, it is often desired to measure the statistical significance of the difference between them. Write a function that takes the CV results of two models, and returns the statistical significance (p-value) that one is better than the other.
* Hint: use a t-test

In [ ]:
!conda create -n env37 python=3.7
y
!conda install -c conda-forge catboost=0.21
y

In [17]:
import time

catboost_onehot2_score = []
catboost_onehot4_score = []
catboost_without_cat_score = []

cat_features_indices = non_float_feat

params = {
    'custom_loss': 'TotalF1:average=Macro',
    'class_weights': label_freq,
    'eval_metric': 'TotalF1:average=Macro',
    'random_seed': 42,
    'verbose': 500,
#     'logging_level': 'Silent',
    'use_best_model': True,
    'one_hot_max_size': 2,
    'cat_features': cat_features_indices,
    'task_type': 'GPU',
    'devices': '0:1'
}

start = time.time()
model_onehot2 = CatBoostClassifier(**params)

rnd = np.random.randint(0, 10000)
X_train, X_valid, y_train, y_valid = train_test_split(train_set, train_labels, test_size=0.3, random_state=rnd)
model_onehot2.fit(X_train, y_train, eval_set=(X_valid, y_valid))
model_onehot2.best_score_.get('validation').get('TotalF1:use_weights=false')

end = time.time()
print(end - start)

Learning rate set to 0.113234


CatBoostError: catboost/cuda/cuda_lib/cuda_manager.cpp:201: Condition violated: `State == nullptr'

In [ ]:
model_onehot2.fit(X_train, y_train, eval_set=(X_valid, y_valid))
model_onehot2.best_score_.get('validation').get('TotalF1:use_weights=false')

In [ ]:
onehot2_score = []
onehot4_score = []
without_cat_score = []

params = {
    'custom_loss': 'TotalF1:average=Macro',
    'class_weights': label_freq,
    'eval_metric': 'TotalF1:average=Macro',
    'random_seed': 42,
    'logging_level': 'Silent',
    'use_best_model': True,
    'one_hot_max_size': 2,
    'cat_features': cat_features_indices
}

model_onehot2 = CatBoostClassifier(**params)
params.update({'one_hot_max_size': 4})
model_onehot4 = CatBoostClassifier(**params)
params.update({'cat_features': None})
model_without_cat = CatBoostClassifier(**params)

for i, random_state in enumerate(random.randint(0, 10000, size=100):
    X_train, X_valid, y_train, y_valid = train_test_split(train_set, train_labels, test_size=0.3, random_state=random_state)
    
    model_onehot2.fit(X_train, y_train, eval_set=(X_valid, y_valid))
    onehot2_score.append(model_onehot2.best_score_.get('validation').get('TotalF1:use_weights=false'))
    
    model_onehot4.fit(X_train, y_train, eval_set=(X_valid, y_valid))
    onehot4_score.append(model_onehot4.best_score_.get('validation').get('TotalF1:use_weights=false'))
    
    model_without_cat.fit(X_train, y_train, eval_set=(X_valid, y_valid))
    without_cat_score.append(model_without_cat.best_score_.get('validation').get('TotalF1:use_weights=false'))
    
    print(f'Iteration {i}')
    print(f'f1-score of onehot=2 / onehot=4 / without-cat models: {onehot2_score[i]:.2f} / {onehot4_score[i]:.2f} / {without_cat_score[i]:.2f}')
    print('-'*30)
                                 

In [ ]:
#compute the total number of data points
n = len(train_labels)
#Compute the difference between the results
diff = [y - x for y, x in zip(valid_scores_with_cat4, valid_scores_with_cat2)]
#Comopute the mean of differences
d_bar = np.mean(diff)
#compute the variance of differences
sigma2 = np.var(diff)
#compute the number of data points used for training 
n1 = 4/5*n
#compute the number of data points used for testing 
n2 = 1/5*n
#compute the modified variance
sigma2_mod = sigma2 * (1/n + n2/n1)
#compute the t_static
t_static =  d_bar / np.sqrt(sigma2_mod)
from scipy.stats import t
#Compute p-value and plot the results 
Pvalue = ((1 - t.cdf(t_static, n-1))*200)
Pvalue

In [ ]:
diff

Use this function to test the improvements of your different models. 

What does this tell you about the importance of the variance of a model's results (and not only its average performance)?


In [ ]:
# your code here

## Hyperparameter tuning
Hyperparameter tuning is an important part of training ML models. Before continuing, read the following article that gives an overview of the popular hyperparameter tuning methods: [Hyperparameter Tuning - O'Reilly Media](../../Hyperparameter Tuning - O'Reilly Media.pdf) 

Throughout this part of the execise, keep in mind that you have limited time and resources. Therefore, you must plan your experiments carefully and not just search over a large hyperparameter space using brute force. 

Document your progress and explain the decisions you made along the way.

### Grid Search
In this section, instead of considering a large grid of hyperparameters, try to think which hyperparameters are the most important, and tune them in an iterative manner. Start with the most important parameters and then move to controlling the less important ones. Try to understand which interval of values is best for them by initially searching in a coarse grained manner and then choose a more fine grained search space depending on the previous results. For example, when tuning for the optimal learning rate, you can at first try different powers of 10 (0.001, 0.01, 0.1) and then, observing that the best results were for 0.01, perform a second search on a smaller interval, such as (0.005, 0.01, 0.05). 

You might find it useful to visualize the interactions between different hyperparameters. Sklearn's `GridSearchCV` class has the attribute ```cv_results_``` which contains the evaluation results of the models trained with the different parameters. The following function allows to plot some results nicely:

In [ ]:
def plot_groups(cv_results, groupby, plotby, plot_train=False, **subplots_params):
    df = pd.DataFrame(cv_results)
    for k in df.params.values[0].keys():
        df[k] = df[['params']].applymap(lambda x: x[k])
    fig, axes = plt.subplots(squeeze=False, **subplots_params)
    for (key, key_group), ax in zip(df.groupby(groupby), axes.flat):
        sorted_group = key_group.sort_values(by=[plotby])
        best_plotby = key_group[plotby].values[np.argmax(key_group.mean_test_score.values)]
        best_score = key_group.mean_test_score.max()
        ax.errorbar(sorted_group[plotby], sorted_group.mean_test_score, yerr=sorted_group.std_test_score)
        if plot_train:
            ax.errorbar(sorted_group.iterations, sorted_group.mean_train_score, yerr=sorted_group.std_train_score)
        ax.set_title(f"{groupby}={key}, best={best_score:.3f}, {best_plotby} {plotby}")

Usage example:

say we performed grid search over the follwing grid (other parameters omitted): 

```python
param_grid = [{'learning_rate': [0.01, 0.005], 'iterations': [600, 900, 1200, 1500]}, {'learning_rate': [0.03, 0.1], 'iterations': [400, 600, 900]}]
gridcv = GridSearchCV(..., param_grid=param_grid)
gridcv.fit(...)
```

We can now use our function to plot the performance of different learning rate as a function of the number of iterations (you can also group by multiple hyperparameters by passing a list of hyperparameter names instead of a single one):
```python
plot_groups(gridcv.cv_results_, groupby='learning_rate', plotby='iterations', nrows=2, ncols=2, sharey=True, figsize=(14,6))
```

The output we will get will be something like this (the results are fictitious):
![alt text](plot_example.png "plot_example.png")

Remember that for sklearn's multiprocessing to work you need to run the search in a .py script. In this case you can just the script and save the results to a csv file, load them here and present them using the function above. 

* **Bonus Note**: To find the optimal number of iterations (number of trees), one has two options. The first, which is the most straighforward, is to treat it as a hyperparameter like all others and tune for it. The second makes use of the `staged_predict` function, which allows prediction using only the first `k` iterations. In this case we will first find the maximum number of iterations we want to check, train with this value in all subsequent models, but assess their performance using the best iteration, which can be found using `staged_predict`. While the second method is more efficient, it doesn't come out-of-the-box with sklearn/catboost, so you can stick with the first method. 

Happy tuning :)

In [ ]:
# your code here

### Random Search
After getting a sense of the hyperparameter space using grid search, try running some random searches and see if you can improve your results. Recall that for any distribution over a sample space with a finite maximum, the maximum of 60 random
observations lies within the top 5% of the true maximum, with 95% probability. You might want to define your sampling distribution based on your observations from the previous section.

In [ ]:
# your code here

### Optional: Bayesian hyperparameter optimization using the hyperopt library
Consult with your mentor whether you should learn about bayesian hyperparameter optimization or not. If you decide you should, the `hyperopt` library is a popular choice on Kaggle competitions and seems to work well.

Definitely add a short `hyperopt` guide/refernce to this exercise if you end up learning it thoroughly and find it valuable!  

In [ ]:
# your code here

## Submission to Kaggle
After you found the best catboost model, you are encouraged to submit it to the Kaggle competition (late submission). Just go to the competition's web page and follow the submission instructions. Remember to train your model on the entire training set when submitting, as you want the highest performance you can get. You can also try using an **ensemble of models** (if you are not familiar with ensembles yet, look it up) trained with different random seeds to improve your score even more :)

## The End.